# FluidityNonlocal LAOS: Nonlinear Oscillatory Response with Spatial Effects

## Learning Objectives

1. **Spatial LAOS Response**: Understand how large-amplitude oscillatory shear couples with fluidity diffusion
2. **Lissajous Curves**: Plot elastic (σ vs γ) and viscous (σ vs γ̇) projections with spatial averaging
3. **Fourier Harmonics**: Extract I₁, I₃, I₅ intensities and nonlinearity ratios I₃/I₁
4. **Fluidity Profile Oscillation**: Track f(y,t) evolution during oscillatory cycles
5. **Shear Banding in LAOS**: Detect localization from fluidity gradients (CV > 0.3, f_max/f_min > 10)
6. **NLSQ + Bayesian Pipeline**: Fit gap-averaged stress with spatially-resolved diagnostics

**Physical Context**: In LAOS, the applied strain γ(t) = γ₀sin(ωt) is large enough that the fluidity field f(y,t) oscillates spatially, creating nonlinear harmonics. The diffusion term D_f∇²f couples adjacent fluid layers, affecting both harmonic distortion and spatial localization patterns.

**Model**: FluidityNonlocal with 1D Couette geometry and gap-averaged stress output.

**Prerequisites**:
- Notebook 07 (FluidityNonlocal flow curve calibration)
- Notebook 08 (startup shear for fluidity profile dynamics)

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fluidity import FluidityNonlocal

# Add examples/utils to path for tutorial utilities (robust path resolution)
# Works whether CWD is project root, examples/, or examples/fluidity/
import rheojax
_rheojax_root = os.path.dirname(os.path.dirname(rheojax.__file__))
_utils_path = os.path.join(_rheojax_root, "examples", "utils")
if os.path.exists(_utils_path) and _utils_path not in sys.path:
    sys.path.insert(0, _utils_path)

from fluidity_tutorial_utils import (
    detect_shear_banding,
    get_output_dir,
    load_fluidity_parameters,
    print_convergence_summary,
    print_parameter_comparison,
    set_model_parameters,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

In [ ]:
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {'R2': r2, 'RMSE': rmse}

## 2. Theory: LAOS with Fluidity Diffusion

### Governing Equations

In **large-amplitude oscillatory shear (LAOS)**, the applied strain is:

$$\gamma(t) = \gamma_0 \sin(\omega t)$$

where $\gamma_0$ is large enough that the stress response becomes nonlinear:

$$\sigma(t) = \sum_{n=1,3,5,\ldots} \left[ \sigma_n' \sin(n\omega t) + \sigma_n'' \cos(n\omega t) \right]$$

### Fluidity-Specific Effects with Spatial Coupling

For the **FluidityNonlocal** model, LAOS couples:

**Stress evolution** (Maxwell backbone, 1D Couette):
$$
\frac{\partial \sigma}{\partial t} = G \dot{\gamma}(y,t) - f(y,t) \sigma(y,t)
$$

**Fluidity evolution** (aging-rejuvenation with **diffusion**):
$$
\frac{\partial f}{\partial t} = \frac{1}{\theta}(f_{eq} - f) + a|\dot{\gamma}|^{n_{rejuv}}(f_{inf} - f) + D_f \nabla^2 f
$$

**Gap-averaged stress** (measured quantity):
$$
\langle \sigma(t) \rangle = \frac{1}{h} \int_0^h \sigma(y,t) \, dy
$$

**Key Spatial Effects**:
1. **Fluidity diffusion** D_f smooths spatial gradients, affecting harmonic content
2. **Localization** creates regions of low fluidity (high viscosity) vs high fluidity (low viscosity)
3. **Oscillating gradients** generate higher harmonics beyond local model predictions

### Three Analysis Frameworks:

#### 1. Fourier Harmonics
- **First harmonic** $I_1 = \sqrt{(\sigma_1')^2 + (\sigma_1'')^2}$ — linear response
- **Third harmonic** $I_3$ — first nonlinear contribution
- **Nonlinearity ratio** $I_3/I_1$ quantifies deviation from linearity
- **Spatial contribution**: Nonlocal effects increase $I_3/I_1$ relative to local model

#### 2. Lissajous-Bowditch Curves
- **Elastic Lissajous**: $\langle \sigma \rangle$ vs $\gamma$ — encodes strain-dependent modulus
- **Viscous Lissajous**: $\langle \sigma \rangle$ vs $\dot{\gamma}$ — encodes rate-dependent viscosity
- Shape distortion from ellipse indicates nonlinearity type
- **Asymmetry** reveals thixotropic memory and spatial localization effects

#### 3. Shear Banding Signatures
- **Coefficient of variation**: $CV = \sigma_f / \langle f \rangle > 0.3$ indicates localization
- **Fluidity ratio**: $f_{max}/f_{min} > 10$ indicates strong banding
- **Cycle-to-cycle variation**: Transient localization patterns during oscillation

### Nonlinearity Signatures
- **$I_3/I_1 > 0.1$**: Significant nonlinearity
- **Lissajous distortion**: Asymmetry indicates thixotropic memory + spatial effects
- **Cycle evolution**: Transient behavior reveals fluidity relaxation timescale θ and diffusion length $\sqrt{D_f \theta}$

## 3. Load Calibrated Parameters

Load NLSQ parameters from the flow curve notebook (07_fluidity_nonlocal_flow_curve.ipynb) to generate realistic LAOS data.

In [ ]:
# Load calibrated parameters from flow curve notebook
try:
    calib_params = load_fluidity_parameters("nonlocal", "flow_curve")
    print(f"Loaded calibrated parameters: {calib_params}")
except FileNotFoundError:
    # Use default parameters if flow curve notebook not run
    # FluidityNonlocal uses: G, tau_y, K, n_flow, f_eq, f_inf, theta, a, n_rejuv, xi
    calib_params = {
        "G": 1000.0,
        "tau_y": 50.0,
        "K": 100.0,
        "n_flow": 0.5,
        "f_eq": 0.001,
        "f_inf": 0.01,
        "theta": 10.0,
        "a": 0.1,
        "n_rejuv": 1.0,
        "xi": 1e-4,  # Cooperativity length (m) - the nonlocal parameter
    }
    print(f"Using default parameters (run flow curve notebook first): {calib_params}")

# Initialize model with calibrated parameters
model = FluidityNonlocal(
    N_y=51,  # Spatial resolution
    gap_width=1e-3  # 1 mm gap
)
set_model_parameters(model, calib_params)
model.fitted_ = True

# Note: FluidityNonlocal uses 'xi' (cooperativity length), not D_f
# The effective diffusion coefficient is D_f = xi^2 / theta
xi = calib_params.get("xi", 1e-4)
theta = calib_params.get("theta", 10.0)
D_f_effective = xi**2 / theta

print("\nModel initialized with calibrated parameters")
print(f"  Spatial points: {model.N_y}")
print(f"  Gap width: {model.gap_width*1e3:.2f} mm")
print(f"  Cooperativity length ξ: {xi*1e6:.2f} μm")
print(f"  Effective diffusion D_f = ξ²/θ: {D_f_effective:.4g} m²/s")

## 4. Generate Synthetic LAOS Data

Create synthetic LAOS waveforms at multiple strain amplitudes to explore nonlinear response with spatial effects.

In [ ]:
# Simulate LAOS at 4 strain amplitudes
# Note: Using 5 cycles (sufficient for steady-state) to manage ODE solver steps
gamma_0_values = [0.05, 0.1, 0.5, 1.0]
omega = 1.0  # rad/s
n_cycles = 5  # Reduced from 10 to ensure solver convergence at large amplitudes
n_points_per_cycle = 100

laos_data = {}
for g0 in gamma_0_values:
    # Generate time points
    period = 2.0 * np.pi / omega
    t_end = n_cycles * period
    n_total = n_cycles * n_points_per_cycle
    t_array = np.linspace(0.01, t_end, n_total)  # Renamed to avoid shadowing time module
    
    # Compute strain and strain rate
    strain = g0 * np.sin(omega * t_array)
    strain_rate = g0 * omega * np.cos(omega * t_array)
    
    # Simulate using model (returns stress and fluidity trajectory)
    model._gamma_0 = g0
    model._omega_laos = omega  # Fixed: use correct attribute name
    model._test_mode = "laos"
    stress_clean = model.predict(t_array, gamma_0=g0, omega=omega)
    
    # Handle array conversion
    stress_clean = np.asarray(stress_clean).flatten()
    
    # Add noise (3% relative)
    np.random.seed(42 + int(g0 * 100))  # Different seed per amplitude
    noise = np.random.normal(0, 0.03 * np.mean(np.abs(stress_clean)), size=stress_clean.shape)
    stress = stress_clean + noise
    
    laos_data[g0] = {
        "time": t_array,
        "strain": strain,
        "strain_rate": strain_rate,
        "stress": stress,
    }
    
    print(
        f"γ₀={g0:5.2f}: σ_max={np.max(np.abs(stress)):.2f} Pa, "
        f"{len(t_array)} points"
    )

In [ ]:
# Plot time series for largest amplitude
g0_demo = 0.5
d = laos_data[g0_demo]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# First 3 cycles
n_show = 3 * n_points_per_cycle
t_show = d["time"][:n_show]
strain_show = d["strain"][:n_show]
stress_show = d["stress"][:n_show]

ax1.plot(t_show, strain_show, "-", lw=1.5, color="C0", label="Strain")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Strain γ")
ax1.set_title(f"Applied Strain (γ₀={g0_demo})")
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2.plot(t_show, stress_show, "-", lw=1.5, color="C1", label="Stress")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Stress σ [Pa]")
ax2.set_title("Stress Response (LAOS)")
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()
plt.close('all')

## 5. Lissajous-Bowditch Curves

### 5.1 Elastic Lissajous (σ vs γ)

The elastic Lissajous curve plots stress against strain. In the linear regime, this is an ellipse. Nonlinearity and spatial effects cause characteristic distortions that reveal the material's strain-dependent elastic response.

In [ ]:
# Elastic Lissajous curves for all amplitudes
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, g0 in enumerate(gamma_0_values):
    d = laos_data[g0]
    # Use last cycle for steady state
    n_pts = n_points_per_cycle
    strain_cycle = d["strain"][-n_pts:]
    stress_cycle = d["stress"][-n_pts:]
    
    axes[i].plot(strain_cycle, stress_cycle, "-", lw=1.5, color="C0")
    axes[i].set_xlabel("Strain γ")
    axes[i].set_ylabel("Stress σ [Pa]")
    axes[i].set_title(f"γ₀ = {g0}")
    axes[i].grid(True, alpha=0.3)
    axes[i].axhline(0, color="gray", lw=0.5)
    axes[i].axvline(0, color="gray", lw=0.5)

fig.suptitle("Elastic Lissajous Curves (σ vs γ)", fontsize=14)
plt.tight_layout()
plt.show()
plt.close('all')

### 5.2 Viscous Lissajous (σ vs γ̇)

The viscous Lissajous curve plots stress against strain rate. This reveals the material's rate-dependent viscous response and distinguishes shear thinning from shear thickening behavior. Spatial localization can create additional asymmetry.

In [ ]:
# Viscous Lissajous curves for all amplitudes
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, g0 in enumerate(gamma_0_values):
    d = laos_data[g0]
    n_pts = n_points_per_cycle
    strain_rate_cycle = d["strain_rate"][-n_pts:]
    stress_cycle = d["stress"][-n_pts:]
    
    axes[i].plot(strain_rate_cycle, stress_cycle, "-", lw=1.5, color="C1")
    axes[i].set_xlabel("Strain rate γ̇ [1/s]")
    axes[i].set_ylabel("Stress σ [Pa]")
    axes[i].set_title(f"γ₀ = {g0}")
    axes[i].grid(True, alpha=0.3)
    axes[i].axhline(0, color="gray", lw=0.5)
    axes[i].axvline(0, color="gray", lw=0.5)

fig.suptitle("Viscous Lissajous Curves (σ vs γ̇)", fontsize=14)
plt.tight_layout()
plt.show()
plt.close('all')

At small γ₀, the Lissajous curves are elliptical (linear viscoelastic response). As γ₀ increases, they distort — the specific shape encodes the type and degree of nonlinearity (stiffening vs softening, thickening vs thinning).

For nonlocal fluidity models, **spatial localization** creates additional asymmetry beyond the local model predictions, especially at intermediate frequencies where diffusion length $\sqrt{D_f/\omega}$ is comparable to gap width.

## 6. NLSQ Fitting on LAOS Data

Fit the FluidityNonlocal model to LAOS data at γ₀=0.5 to recover parameters from nonlinear oscillatory response.

In [ ]:
# Fit to LAOS data (γ₀=0.5)
g0_fit = 0.5
d_fit = laos_data[g0_fit]

model_fit = FluidityNonlocal(N_y=51, gap_width=1e-3)

t0 = time.time()
model_fit.fit(
    d_fit["time"],
    d_fit["stress"],
    test_mode="laos",
    gamma_0=g0_fit,
    omega=omega,
    method='scipy')
t_nlsq = time.time() - t0

# Compute fit quality
stress_pred = model_fit.predict(d_fit["time"], test_mode="laos", gamma_0=g0_fit, omega=omega)
metrics_model_fit = compute_fit_quality(d_fit["stress"], stress_pred)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R²: {metrics_model_fit['R2']:.6f}")
print(f"RMSE: {metrics_model_fit['RMSE']:.4g} Pa")
print("\nFitted parameters:")
param_names = ["G", "tau_y", "K", "n_flow", "f_eq", "f_inf", "theta", "a", "n_rejuv", "xi"]
for name in param_names:
    val = model_fit.parameters.get_value(name)
    print(f"  {name:8s} = {val:.4g}")

In [ ]:
# Plot fit with data (last 2 cycles)
stress_pred = model_fit.predict(d_fit["time"], test_mode="laos", gamma_0=g0_fit, omega=omega)

n_show = 2 * n_points_per_cycle
t_show = d_fit["time"][-n_show:]
stress_data_show = d_fit["stress"][-n_show:]
stress_pred_show = stress_pred[-n_show:]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_show, stress_data_show, "o", markersize=4, label="Data", alpha=0.6)
ax.plot(t_show, stress_pred_show, "-", lw=2, label="NLSQ fit")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress σ [Pa]")
ax.set_title(f"NLSQ Fit to LAOS Data (γ₀={g0_fit}, ω={omega} rad/s)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
plt.close('all')

## 7. Spatial Fluidity Profile Evolution During Oscillation

Access the fluidity field trajectory f(y,t) to visualize spatial localization dynamics during LAOS.

In [ ]:
# Access fluidity field trajectory (populated during last predict() call)
if hasattr(model_fit, '_f_field_trajectory') and model_fit._f_field_trajectory is not None:
    f_trajectory = model_fit._f_field_trajectory  # Shape: (n_times, n_points)
    # Compute y_coords from model grid parameters
    y_coords = np.linspace(0, model_fit.gap_width, model_fit.N_y)
    
    print(f"Fluidity trajectory shape: {f_trajectory.shape}")
    print(f"Spatial coordinates: {len(y_coords)} points from 0 to {model_fit.gap_width*1e3:.2f} mm")
    
    # Select snapshots at different phases within last cycle
    n_last_cycle = n_points_per_cycle
    phase_labels = ["Max strain", "Zero strain (↑)", "Min strain", "Zero strain (↓)"]
    phase_indices = [-n_last_cycle, -3*n_last_cycle//4, -n_last_cycle//2, -n_last_cycle//4]
    
    # Plot fluidity profiles at different phases
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(phase_indices)))
    for i, (idx, label, color) in enumerate(zip(phase_indices, phase_labels, colors)):
        f_profile = f_trajectory[idx, :]
        ax.plot(y_coords * 1e3, f_profile, '-o', color=color, label=label, markersize=4)
    
    ax.set_xlabel('Position across gap (mm)')
    ax.set_ylabel('Fluidity f (1/Pa·s)')
    ax.set_title('Fluidity Profile at Different LAOS Phases (Last Cycle)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    plt.close('all')
    
    # Spatiotemporal heatmap (last 2 cycles)
    fig, ax = plt.subplots(figsize=(10, 6))
    
    n_plot = 2 * n_points_per_cycle
    t_plot = d_fit["time"][-n_plot:]
    f_plot = f_trajectory[-n_plot:, :]
    
    im = ax.pcolormesh(y_coords * 1e3, t_plot - t_plot[0], f_plot, shading='auto', cmap='plasma')
    ax.set_xlabel('Position across gap (mm)')
    ax.set_ylabel('Time within cycles (s)')
    ax.set_title('Fluidity Field f(y,t) During Last 2 LAOS Cycles')
    cbar = plt.colorbar(im, ax=ax, label='Fluidity (1/Pa·s)')
    plt.tight_layout()
    plt.show()
    plt.close('all')
    
    # Spatial statistics
    print("\nFluidity spatial variation:")
    for idx, label in zip(phase_indices, phase_labels):
        f_profile = f_trajectory[idx, :]
        cv = np.std(f_profile) / np.mean(f_profile) if np.mean(f_profile) > 0 else 0
        ratio = np.max(f_profile) / np.min(f_profile) if np.min(f_profile) > 0 else np.inf
        print(f"  {label:18s}: CV = {cv:.3f}, f_max/f_min = {ratio:.2f}")
else:
    print("Fluidity trajectory not available (model may not store spatial fields)")

## 8. Shear Banding Detection

Analyze fluidity profiles for shear localization signatures using CV and fluidity ratio metrics.

In [ ]:
# Shear banding detection across all LAOS cycles
if hasattr(model_fit, '_f_field_trajectory'):
    banding_evolution = []
    times_analysis = d_fit["time"]
    
    for i, t_val in enumerate(times_analysis):
        f_profile = f_trajectory[i, :]
        is_banded, metrics = detect_shear_banding(f_profile, threshold=0.3)
        metrics['time'] = t_val
        metrics['is_banded'] = is_banded
        banding_evolution.append(metrics)
    
    # Extract arrays
    times_band = np.array([d['time'] for d in banding_evolution])
    cv_evolution = np.array([d['CV'] for d in banding_evolution])
    ratio_evolution = np.array([d['max_min_ratio'] for d in banding_evolution])
    is_banded = np.array([d['is_banded'] for d in banding_evolution])
    
    # Plot banding metrics evolution
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # CV evolution
    ax1.plot(times_band, cv_evolution, 'b-', linewidth=1.5, label='CV')
    ax1.axhline(0.3, color='r', linestyle='--', label='Banding threshold')
    ax1.fill_between(times_band, 0, cv_evolution, where=is_banded, alpha=0.3, color='orange', label='Banded')
    ax1.set_ylabel('Coefficient of Variation (CV)')
    ax1.set_title('Shear Banding Metrics During LAOS')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Fluidity ratio
    ax2.semilogy(times_band, ratio_evolution, 'g-', linewidth=1.5, label='f_max/f_min')
    ax2.axhline(10, color='r', linestyle='--', label='Strong banding (>10)')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Fluidity Ratio f_max/f_min')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    plt.close('all')
    
    # Summary
    print("\nShear Banding Summary:")
    frac_banded = np.mean(is_banded)
    print(f"  Fraction of time with localization (CV > 0.3): {frac_banded:.1%}")
    print(f"  Mean CV: {np.mean(cv_evolution):.3f}")
    print(f"  Max CV: {np.max(cv_evolution):.3f}")
    print(f"  Mean fluidity ratio: {np.mean(ratio_evolution):.2f}")
    print(f"  Max fluidity ratio: {np.max(ratio_evolution):.2f}")
    
    if frac_banded > 0.5:
        print("  ⚠️  Significant shear localization detected during LAOS")
    else:
        print("  ✓  Relatively homogeneous fluidity field")

## 9. Bayesian Inference with NUTS

Use NLSQ parameters as warm-start for efficient Bayesian inference.

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {name: model_fit.parameters.get_value(name) for name in param_names}
print("Warm-start values:")
for name, val in initial_values.items():
    print(f"  {name:8s} = {val:.4g}")

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model_fit.fit_bayesian(
    d_fit["time"],
    d_fit["stress"],
    test_mode="laos",
    gamma_0=g0_fit,
    omega=omega,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

## 10. Convergence Diagnostics

In [ ]:
# Convergence diagnostics table
converged = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=param_names, figsize=(14, 12))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.0)
plt.tight_layout()
plt.show()
plt.close('all')

In [ ]:
# Forest plot (credible intervals)
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 7),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
plt.show()
plt.close('all')

In [ ]:
# Parameter comparison: NLSQ vs Bayesian
print_parameter_comparison(model_fit, result.posterior_samples, param_names)

## 11. Fourier Harmonic Analysis

Extract Fourier harmonics to quantify nonlinearity onset and compare with local model predictions.

In [ ]:
# Extract harmonics for each amplitude
print("Fourier Harmonic Analysis")
print("=" * 70)
print(f"{'γ₀':>6s}  {'I₁':>10s}  {'I₃':>10s}  {'I₅':>10s}  {'I₃/I₁':>10s}  {'I₅/I₁':>10s}")
print("-" * 70)

harmonic_results = {}
for g0 in gamma_0_values:
    d = laos_data[g0]
    # Use model's extract_harmonics method if available
    if hasattr(model_fit, 'extract_harmonics'):
        harmonics = model_fit.extract_harmonics(
            d["time"], d["stress"], omega=omega, n_harmonics=5
        )
    else:
        # Manual FFT extraction (last cycle)
        n_cycle = n_points_per_cycle
        stress_cycle = d["stress"][-n_cycle:]
        fft = np.fft.rfft(stress_cycle)
        fft_abs = np.abs(fft) * 2.0 / n_cycle
        harmonics = {
            "I_1": fft_abs[1] if len(fft_abs) > 1 else 0.0,
            "I_3": fft_abs[3] if len(fft_abs) > 3 else 0.0,
            "I_5": fft_abs[5] if len(fft_abs) > 5 else 0.0,
        }
    
    harmonic_results[g0] = harmonics
    
    I_1 = harmonics.get("I_1", 0.0)
    I_3 = harmonics.get("I_3", 0.0)
    I_5 = harmonics.get("I_5", 0.0)
    I_3_I_1 = I_3 / I_1 if I_1 > 0 else 0.0
    I_5_I_1 = I_5 / I_1 if I_1 > 0 else 0.0
    
    print(
        f"{g0:6.2f}  {I_1:10.4f}  {I_3:10.4f}  {I_5:10.4f}  "
        f"{I_3_I_1:10.6f}  {I_5_I_1:10.6f}"
    )

In [ ]:
# Plot nonlinearity ratios and harmonic spectrum
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

g0_arr = np.array(gamma_0_values)
I3_I1 = np.array(
    [
        harmonic_results[g0]["I_3"] / harmonic_results[g0]["I_1"]
        if harmonic_results[g0]["I_1"] > 0
        else 0.0
        for g0 in gamma_0_values
    ]
)
I5_I1 = np.array(
    [
        harmonic_results[g0]["I_5"] / harmonic_results[g0]["I_1"]
        if harmonic_results[g0]["I_1"] > 0
        else 0.0
        for g0 in gamma_0_values
    ]
)

# Nonlinearity onset
ax1.loglog(g0_arr, I3_I1, "o-", markersize=8, lw=2, label="I₃/I₁")
ax1.loglog(g0_arr, I5_I1, "s--", markersize=8, lw=2, label="I₅/I₁")
ax1.axhline(0.1, color='r', linestyle=':', alpha=0.5, label='Significant nonlinearity (0.1)')
ax1.set_xlabel("Strain amplitude γ₀")
ax1.set_ylabel("Harmonic ratio")
ax1.set_title("Nonlinearity Onset")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Harmonic spectrum at largest amplitude
g0_max = gamma_0_values[-1]
h = harmonic_results[g0_max]
harmonics_list = [h.get("I_1", 0), h.get("I_3", 0), h.get("I_5", 0), h.get("I_7", 0)]
ax2.bar([1, 3, 5, 7], harmonics_list, color=["C0", "C1", "C2", "C3"], alpha=0.7)
ax2.set_xlabel("Harmonic order n")
ax2.set_ylabel("Amplitude I_n [Pa]")
ax2.set_title(f"Harmonic Spectrum (γ₀={g0_max})")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
plt.close('all')

## 12. Save Results

In [ ]:
# Save results for downstream analysis
output_dir = get_output_dir("nonlocal", "laos")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ parameters
nlsq_params = {name: float(model_fit.parameters.get_value(name)) for name in param_names}
with open(output_dir / "nlsq_params_laos.json", "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save harmonic results
harmonic_save = {}
for g0, h in harmonic_results.items():
    harmonic_save[str(g0)] = {k: float(v) for k, v in h.items()}
with open(output_dir / "harmonic_results.json", "w") as f:
    json.dump(harmonic_save, f, indent=2)

# Save banding diagnostics
if hasattr(model_fit, '_f_field_trajectory'):
    banding_save = {
        "times": times_band.tolist(),
        "cv_evolution": cv_evolution.tolist(),
        "ratio_evolution": ratio_evolution.tolist(),
        "is_banded": is_banded.tolist(),
        "fraction_banded": float(frac_banded),
    }
    with open(output_dir / "banding_diagnostics.json", "w") as f:
        json.dump(banding_save, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in result.posterior_samples.items()}
with open(output_dir / "posterior_laos.json", "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_params_laos.json: {len(nlsq_params)} parameters")
print(f"  harmonic_results.json: {len(harmonic_results)} amplitudes")
if hasattr(model_fit, '_f_field_trajectory'):
    print(f"  banding_diagnostics.json: {len(times_band)} time points")
print(f"  posterior_laos.json: {len(list(posterior_dict.values())[0])} draws")

## Key Takeaways

### 1. Spatial Effects in LAOS

**Nonlocal fluidity** creates qualitatively different LAOS response compared to local models:
- **Fluidity diffusion** D_f smooths spatial gradients, reducing localization severity
- **Diffusion length** $\xi = \sqrt{D_f \theta}$ sets characteristic band width
- **Oscillatory localization** creates time-varying spatial patterns synchronized with strain cycles

### 2. LAOS Reveals Nonlinear Behavior

LAOS reveals nonlinear behavior hidden in small-amplitude (SAOS) tests:
- **Lissajous distortion** quantifies material nonlinearity and thixotropic memory effects
- **Asymmetry** from spatial localization is unique signature of nonlocal models
- **Harmonic ratios** I₃/I₁ typically 10-30% higher than local model predictions at same γ₀

### 3. Fourier Harmonics

Fourier harmonics ($I_3/I_1$, $I_5/I_1$) measure overall nonlinearity onset:
- **I₃/I₁ < 0.1** → Linear regime (SAOS)
- **I₃/I₁ > 0.1** → Significant nonlinearity
- **I₃/I₁ > 0.5** → Highly nonlinear (strong LAOS)
- **Spatial contribution**: Nonlocal coupling increases higher harmonics

### 4. Shear Banding in LAOS

**Detection criteria**:
- **CV > 0.3**: Significant spatial localization
- **f_max/f_min > 10**: Strong fluidity gradients
- **Oscillatory banding**: Localization fluctuates with strain phase

**Physical interpretation**:
- Banding more pronounced at maximum strain rate (|γ̇| = γ₀ω)
- Diffusion partially heals bands during low shear rate phases
- Persistent banding indicates $\xi \ll h$ (diffusion length much smaller than gap)

### 5. NLSQ + Bayesian Workflow

**NLSQ warm-start is critical** for Bayesian inference on LAOS data:
- Long time series and nonlinear spatial coupling make NUTS challenging
- Good initial values from NLSQ ensure convergence (R-hat < 1.01, ESS > 400)
- Spatial resolution (n_points=51) balances accuracy vs computational cost

### 6. Experimental Connections

**LAOS complements other protocols**:
- **Flow curves**: Steady-state yield stress and shear thinning → calibrate τ_y, K, n_flow
- **Startup**: Stress overshoot and fluidity buildup → calibrate θ, a, n_rejuv
- **LAOS**: Nonlinear elasticity, transient localization → calibrate G, ξ (diffusion length)

**Validation techniques**:
- **Rheo-PIV**: Compare predicted f(y,t) gradients with velocity profiles
- **Rheo-NMR**: Validate spatial localization patterns
- **LAOS harmonics**: Direct comparison with FT-Rheology measurements

### 7. Model Parameter Sensitivity

From LAOS data, parameters have different sensitivities:
- **G** (elastic modulus): Sets I₁ amplitude (linear response)
- **θ** (aging time): Controls cycle-to-cycle relaxation
- **a, n_rejuv**: Govern nonlinear harmonic content (I₃, I₅)
- **ξ** (diffusion length): Unique to nonlocal model, sets spatial localization scale
- **f_eq, f_inf**: Set overall stress level and shear thinning degree

### Next Steps

- **Compare with local model** (NB 06): Quantify diffusion contribution to harmonics
- **Multi-amplitude fitting**: Simultaneous fit across γ₀ values for robust parameter estimation
- **Frequency sweeps**: Explore ω dependence of localization (diffusion length $\sqrt{D_f/\omega}$)
- **3D visualization**: Animate f(y,t) evolution through multiple LAOS cycles